In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import math

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: The Blelloch Algorithm

### Why Work-Efficiency Matters

```
Hillis-Steele: O(n log n) work, O(log n) steps
Blelloch:      O(n) work, O(2 log n) steps

For n = 1 million:
  Hillis-Steele: ~20 million operations
  Blelloch:      ~2 million operations (10x less!)

GPUs have finite compute. Less work = faster execution.
```

### Two-Phase Algorithm

```
Phase 1: UP-SWEEP (Reduce)
  Build partial sums in a tree structure
  After: Last element contains total sum

Phase 2: DOWN-SWEEP
  Distribute partial sums back down
  After: Each element contains its prefix sum
```

### CUDA C++ Implementation (Primary)

In [ ]:
%%writefile blelloch_scan.cu
// blelloch_scan.cu - Work-efficient parallel scan
#include <stdio.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 256
#define LOG_BLOCK_SIZE 8  // log2(256)

// Bank conflict avoidance macro
#define NUM_BANKS 32
#define LOG_NUM_BANKS 5
#define CONFLICT_FREE_OFFSET(n) ((n) >> LOG_NUM_BANKS)

// Blelloch exclusive scan
__global__ void blelloch_scan(int* data, int n) {
    __shared__ int temp[BLOCK_SIZE + BLOCK_SIZE/NUM_BANKS];  // Extra for padding
    
    int tid = threadIdx.x;
    int offset = 1;
    
    // Load with bank conflict avoidance
    int ai = tid;
    int bi = tid + (n/2);
    int bankOffsetA = CONFLICT_FREE_OFFSET(ai);
    int bankOffsetB = CONFLICT_FREE_OFFSET(bi);
    
    temp[ai + bankOffsetA] = data[ai];
    temp[bi + bankOffsetB] = data[bi];
    
    // ========== UP-SWEEP (Reduce) ==========
    for (int d = n >> 1; d > 0; d >>= 1) {
        __syncthreads();
        
        if (tid < d) {
            int ai = offset * (2 * tid + 1) - 1;
            int bi = offset * (2 * tid + 2) - 1;
            ai += CONFLICT_FREE_OFFSET(ai);
            bi += CONFLICT_FREE_OFFSET(bi);
            
            temp[bi] += temp[ai];
        }
        offset *= 2;
    }
    
    // Clear last element (for exclusive scan)
    if (tid == 0) {
        temp[n - 1 + CONFLICT_FREE_OFFSET(n-1)] = 0;
    }
    
    // ========== DOWN-SWEEP ==========
    for (int d = 1; d < n; d *= 2) {
        offset >>= 1;
        __syncthreads();
        
        if (tid < d) {
            int ai = offset * (2 * tid + 1) - 1;
            int bi = offset * (2 * tid + 2) - 1;
            ai += CONFLICT_FREE_OFFSET(ai);
            bi += CONFLICT_FREE_OFFSET(bi);
            
            int t = temp[ai];
            temp[ai] = temp[bi];
            temp[bi] += t;
        }
    }
    __syncthreads();
    
    // Write results
    data[ai] = temp[ai + bankOffsetA];
    data[bi] = temp[bi + bankOffsetB];
}

// Simple version without bank conflict avoidance (easier to understand)
__global__ void blelloch_scan_simple(int* data, int n) {
    __shared__ int temp[BLOCK_SIZE];
    int tid = threadIdx.x;
    
    // Load
    temp[2*tid] = data[2*tid];
    temp[2*tid + 1] = data[2*tid + 1];
    
    // UP-SWEEP
    int offset = 1;
    for (int d = n >> 1; d > 0; d >>= 1) {
        __syncthreads();
        if (tid < d) {
            int ai = offset * (2*tid + 1) - 1;
            int bi = offset * (2*tid + 2) - 1;
            temp[bi] += temp[ai];
        }
        offset *= 2;
    }
    
    // Clear last
    if (tid == 0) temp[n-1] = 0;
    
    // DOWN-SWEEP
    for (int d = 1; d < n; d *= 2) {
        offset >>= 1;
        __syncthreads();
        if (tid < d) {
            int ai = offset * (2*tid + 1) - 1;
            int bi = offset * (2*tid + 2) - 1;
            int t = temp[ai];
            temp[ai] = temp[bi];
            temp[bi] += t;
        }
    }
    __syncthreads();
    
    // Write
    data[2*tid] = temp[2*tid];
    data[2*tid + 1] = temp[2*tid + 1];
}

int main() {
    int h_data[] = {3, 1, 7, 0, 4, 1, 6, 3};
    int n = 8;
    
    printf("Input:    ");
    for (int i = 0; i < n; i++) printf("%d ", h_data[i]);
    printf("\n");
    
    int* d_data;
    cudaMalloc(&d_data, n * sizeof(int));
    cudaMemcpy(d_data, h_data, n * sizeof(int), cudaMemcpyHostToDevice);
    
    blelloch_scan_simple<<<1, n/2>>>(d_data, n);
    
    cudaMemcpy(h_data, d_data, n * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("Exclusive: ");
    for (int i = 0; i < n; i++) printf("%d ", h_data[i]);
    printf("\n");
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o blelloch_scan blelloch_scan.cu
!./blelloch_scan

---

## Part 2: Visualizing Up-Sweep

### Up-Sweep (Reduce Phase)

```
Input: [3, 1, 7, 0, 4, 1, 6, 3]

Step 1 (d=4, offset=1): Add pairs
  temp[1] += temp[0]:  [3, 4, 7, 0, 4, 1, 6, 3]
  temp[3] += temp[2]:  [3, 4, 7, 7, 4, 1, 6, 3]
  temp[5] += temp[4]:  [3, 4, 7, 7, 4, 5, 6, 3]
  temp[7] += temp[6]:  [3, 4, 7, 7, 4, 5, 6, 9]

Step 2 (d=2, offset=2): Add every 4th
  temp[3] += temp[1]:  [3, 4, 7, 11, 4, 5, 6, 9]
  temp[7] += temp[5]:  [3, 4, 7, 11, 4, 5, 6, 14]

Step 3 (d=1, offset=4): Add every 8th
  temp[7] += temp[3]:  [3, 4, 7, 11, 4, 5, 6, 25]

After up-sweep: temp[n-1] = total sum (25)
```

In [ ]:
def visualize_up_sweep(arr):
    """Visualize the up-sweep (reduce) phase."""
    temp = arr.copy().astype(np.int32)
    n = len(temp)
    
    print(f"Input: {temp}")
    print("\n=== UP-SWEEP (Reduce) ===")
    
    offset = 1
    d = n // 2
    step = 1
    
    while d > 0:
        print(f"\nStep {step} (d={d}, offset={offset}):")
        
        for i in range(d):
            ai = offset * (2*i + 1) - 1
            bi = offset * (2*i + 2) - 1
            print(f"  temp[{bi}] += temp[{ai}]: {temp[bi]} + {temp[ai]} = {temp[bi] + temp[ai]}")
            temp[bi] += temp[ai]
        
        print(f"  Result: {temp}")
        
        offset *= 2
        d //= 2
        step += 1
    
    print(f"\nFinal: temp[n-1] = {temp[n-1]} (total sum)")
    return temp

test = np.array([3, 1, 7, 0, 4, 1, 6, 3], dtype=np.int32)
after_upsweep = visualize_up_sweep(test)

---

## Part 3: Visualizing Down-Sweep

### Down-Sweep Phase

```
After up-sweep: [3, 4, 7, 11, 4, 5, 6, 25]

Set last to 0: [3, 4, 7, 11, 4, 5, 6, 0]

Step 1 (d=1, offset=4):
  swap temp[3], temp[7] and add:
  t = temp[3] = 11
  temp[3] = temp[7] = 0
  temp[7] = 0 + 11 = 11
  → [3, 4, 7, 0, 4, 5, 6, 11]

Step 2 (d=2, offset=2):
  For i=0: temp[1], temp[3]
  For i=1: temp[5], temp[7]
  → [3, 0, 7, 4, 4, 11, 6, 16]

Step 3 (d=4, offset=1):
  Process all pairs
  → [0, 3, 4, 11, 11, 15, 16, 22]

Result: Exclusive scan!
```

In [ ]:
def visualize_down_sweep(temp):
    """Visualize the down-sweep phase."""
    temp = temp.copy()
    n = len(temp)
    
    print(f"After up-sweep: {temp}")
    
    # Clear last element
    temp[n-1] = 0
    print(f"Clear last:     {temp}")
    print("\n=== DOWN-SWEEP ===")
    
    offset = n // 2
    d = 1
    step = 1
    
    while d < n:
        print(f"\nStep {step} (d={d}, offset={offset}):")
        
        for i in range(d):
            ai = offset * (2*i + 1) - 1
            bi = offset * (2*i + 2) - 1
            
            t = temp[ai]
            temp[ai] = temp[bi]
            temp[bi] += t
            
            print(f"  swap+add: temp[{ai}]={temp[ai]}, temp[{bi}]={temp[bi]}")
        
        print(f"  Result: {temp}")
        
        offset //= 2
        d *= 2
        step += 1
    
    print(f"\nFinal (Exclusive Scan): {temp}")
    return temp

result = visualize_down_sweep(after_upsweep)

---

## Part 4: Python/Numba Implementation (Optional)

In [ ]:
@cuda.jit
def blelloch_exclusive_scan(data, n):
    """Blelloch work-efficient exclusive scan."""
    temp = cuda.shared.array(256, dtype=np.int32)
    tid = cuda.threadIdx.x
    
    # Each thread loads 2 elements
    if 2*tid < n:
        temp[2*tid] = data[2*tid]
    if 2*tid + 1 < n:
        temp[2*tid + 1] = data[2*tid + 1]
    
    # UP-SWEEP
    offset = 1
    d = n // 2
    while d > 0:
        cuda.syncthreads()
        if tid < d:
            ai = offset * (2*tid + 1) - 1
            bi = offset * (2*tid + 2) - 1
            temp[bi] += temp[ai]
        offset *= 2
        d //= 2
    
    # Clear last for exclusive scan
    if tid == 0:
        temp[n - 1] = 0
    
    # DOWN-SWEEP
    d = 1
    while d < n:
        offset //= 2
        cuda.syncthreads()
        if tid < d:
            ai = offset * (2*tid + 1) - 1
            bi = offset * (2*tid + 2) - 1
            t = temp[ai]
            temp[ai] = temp[bi]
            temp[bi] += t
        d *= 2
    
    cuda.syncthreads()
    
    # Write back
    if 2*tid < n:
        data[2*tid] = temp[2*tid]
    if 2*tid + 1 < n:
        data[2*tid + 1] = temp[2*tid + 1]

In [ ]:
# Test Blelloch scan
def cpu_exclusive_scan(arr):
    result = np.zeros_like(arr)
    for i in range(1, len(arr)):
        result[i] = result[i-1] + arr[i-1]
    return result

test_data = np.array([3, 1, 7, 0, 4, 1, 6, 3], dtype=np.int32)
expected = cpu_exclusive_scan(test_data)

d_data = cuda.to_device(test_data.copy())
blelloch_exclusive_scan[1, 4](d_data, len(test_data))  # n/2 threads
result = d_data.copy_to_host()

print(f"Input:    {test_data}")
print(f"Result:   {result}")
print(f"Expected: {expected}")
print(f"Correct:  {'✓' if np.array_equal(result, expected) else '✗'}")

---

## Part 5: Work Comparison

In [ ]:
def compare_algorithms(n):
    """Compare work done by different scan algorithms."""
    
    # Sequential
    seq_work = n - 1  # n-1 additions
    seq_steps = n - 1
    
    # Hillis-Steele
    hs_steps = int(np.ceil(np.log2(n)))
    hs_work = sum(n - 2**i for i in range(hs_steps))
    
    # Blelloch
    bl_steps = 2 * int(np.ceil(np.log2(n)))  # up + down
    # Up-sweep: n/2 + n/4 + ... + 1 = n-1 additions
    # Down-sweep: same, plus swaps
    bl_work = 2 * (n - 1)
    
    return {
        'Sequential': (seq_work, seq_steps),
        'Hillis-Steele': (hs_work, hs_steps),
        'Blelloch': (bl_work, bl_steps)
    }

print(f"{'N':<10} {'Algorithm':<15} {'Work':<12} {'Steps':<10} {'Work/n':<10}")
print("=" * 60)

for n in [8, 256, 1024, 65536]:
    results = compare_algorithms(n)
    for algo, (work, steps) in results.items():
        print(f"{n:<10} {algo:<15} {work:<12} {steps:<10} {work/n:.2f}")
    print()

---

## Exercises

### Exercise 1: Inclusive Blelloch Scan

In [ ]:
# TODO: Modify Blelloch to produce INCLUSIVE scan
# Hint: Save original values and add them back after exclusive scan

@cuda.jit
def blelloch_inclusive_scan(data, n):
    """Blelloch inclusive scan."""
    pass  # Your implementation

# Expected: [3, 4, 11, 11, 15, 16, 22, 25]

### Exercise 2: Bank Conflict Avoidance

In [ ]:
# TODO: Add bank conflict avoidance padding
# Use CONFLICT_FREE_OFFSET macro pattern

NUM_BANKS = 32
LOG_NUM_BANKS = 5

def conflict_free_offset(n):
    return n >> LOG_NUM_BANKS

# Demonstrate the padding
print("Index mapping with padding:")
for i in range(32):
    padded = i + conflict_free_offset(i)
    bank = padded % 32
    print(f"  index {i:2d} -> padded {padded:2d} (bank {bank})")

---

## Summary

### Blelloch Algorithm

| Property | Value |
|----------|-------|
| Work | O(n) |
| Steps | O(2 log n) |
| Work-efficient | Yes! |
| Output | Exclusive scan |

### CUDA C++ Key Pattern

```cpp
// UP-SWEEP: Build partial sums
for (d = n/2; d > 0; d /= 2) {
    if (tid < d) {
        int ai = offset * (2*tid + 1) - 1;
        int bi = offset * (2*tid + 2) - 1;
        temp[bi] += temp[ai];
    }
    offset *= 2;
}

// Set last to 0
temp[n-1] = 0;

// DOWN-SWEEP: Distribute sums
for (d = 1; d < n; d *= 2) {
    offset /= 2;
    if (tid < d) {
        int ai = offset * (2*tid + 1) - 1;
        int bi = offset * (2*tid + 2) - 1;
        int t = temp[ai];
        temp[ai] = temp[bi];
        temp[bi] += t;
    }
}
```

### Next: Large Array Scan
Tomorrow we'll handle arrays larger than one block can process.